In [6]:
import math

In [7]:
class GeoPolygon():
    def __init__(self,srid=4326):
        self.srid = srid
    
    def from_centroid(self,num_sides, center_latitude, center_longitude, radius_decimal_degrees):
        if num_sides > 2:
            vertices = [None] * (num_sides + 1)
        for i in range(0, num_sides):
            angle_step = 360. / num_sides
            angle_deg = angle_step * i
            angle_rad = math.pi / 180.0 * angle_deg
            y_vert = center_latitude + radius_decimal_degrees * math.sin(angle_rad)
            if abs(y_vert) < 1e-5:
                y_vert = 0
            x_vert = center_longitude + radius_decimal_degrees * math.cos(angle_rad)
            if abs(x_vert) < 1e-5:
                x_vert = 0
            vert = {'x': x_vert, 'y': y_vert}
            vertices[i] = (vert)
            if i == 0:
                vertices[num_sides] = vert
        self.vertices = vertices
        self.center_latitude = center_latitude
        self.center_longitude = center_longitude
        self.radius = radius_decimal_degrees
        
    def to_wkt(self):
        v_str = ''
        for v in self.vertices:
            v_str = v_str + str(v['x']) + ' ' + str(v['y']) + ','
        v_str=v_str[0:-1]
        self.wkt = 'POLYGON(({v_str}))'.format(v_str=v_str)
        
    def to_ewkt(self):
        v_str = ''
        for v in self.vertices:
            v_str = v_str + str(v['x']) + ' ' + str(v['y']) + ','
        v_str=v_str[0:-1]
        self.ewkt = 'SRID={srid};POLYGON(({v_str}))'.format(srid=self.srid,v_str=v_str)


In [36]:
class HexGrid():
    def __init__(self,srid=4326):
        self.srid = srid

    def generate_hex_centroids(self,ullat,ullon,radius_dd,nrows,ncols):
        """

        """
        centers = []
        width = float(radius_dd) * 2
        horiz = width * 3.0/4.0
        height = (math.sqrt(3.0) / 2.0) * width
        vert = height / 2.0
        for i in range(0,ncols):
            x0 = (i * horiz) + float(ullon)
            for j in range(0,nrows):
                if i % 2 == 0:
                    y0 = float(ullat) - (j * vert * 2)
                else:
                    odlat = ullat - vert
                    y0 = float(odlat) - (j * vert * 2)
                centers.append((x0,y0,radius_dd))
        self.hex_centroids = centers
        
    def hex_grid_from_centroids(self):
        hex_grid = []
        p = GeoPolygon()
        for c in self.hex_centroids:
            p.from_centroid(6,c[1],c[0],c[2])
            hex_grid.append(p)
        self.hex_grid = hex_grid
        
    def wkt_grid_from_hexes(self):
        wkt_grid = []
        for h in self.hex_grid:
            h.to_wkt()
            wkt_grid.append(h.wkt)
        self.hex_grid_wkt = wkt_grid 
        
    def ewkt_grid_from_hexes(self):
        ewkt_grid = []
        for h in self.hex_grid:
            h.to_ewkt()
            ewkt_grid.append(h.ewkt)
        self.hex_grid_ewkt = ewkt_grid
        

In [37]:
hg = HexGrid()

In [38]:
hg.generate_hex_centroids(33.0,-120.0,0.5,5,5)

In [39]:
hg.hex_grid_from_centroids()

In [40]:
hg.wkt_grid_from_hexes()

In [41]:
hg.ewkt_grid_from_hexes()

In [42]:
hg.hex_grid_ewkt

['SRID=4326;POLYGON((-116.5 29.535898384862247,-116.75 29.968911086754467,-117.25 29.968911086754467,-117.5 29.535898384862247,-117.25 29.102885682970026,-116.75 29.102885682970026,-116.5 29.535898384862247))',
 'SRID=4326;POLYGON((-116.5 29.535898384862247,-116.75 29.968911086754467,-117.25 29.968911086754467,-117.5 29.535898384862247,-117.25 29.102885682970026,-116.75 29.102885682970026,-116.5 29.535898384862247))',
 'SRID=4326;POLYGON((-116.5 29.535898384862247,-116.75 29.968911086754467,-117.25 29.968911086754467,-117.5 29.535898384862247,-117.25 29.102885682970026,-116.75 29.102885682970026,-116.5 29.535898384862247))',
 'SRID=4326;POLYGON((-116.5 29.535898384862247,-116.75 29.968911086754467,-117.25 29.968911086754467,-117.5 29.535898384862247,-117.25 29.102885682970026,-116.75 29.102885682970026,-116.5 29.535898384862247))',
 'SRID=4326;POLYGON((-116.5 29.535898384862247,-116.75 29.968911086754467,-117.25 29.968911086754467,-117.5 29.535898384862247,-117.25 29.102885682970026,-1